<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Foodtruck deployment in Paris</font></h1>


<h2 align=left><font size = 5>0. Import libraries</font></h2>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


print('Libraries imported.')

Libraries imported.


<h2 align=left><font size = 5>1. Data collection</font></h2>

<h3 align=left><font size = 5>1.1 Paris base statistics</font></h3>

The study will be conducted on regions 75, 92, 93, 94, covering Paris and close suburbs\
The first dataset provides basic statistics on Paris communes

In [2]:
url1 ='https://data.iledefrance.fr/api/records/1.0/search/?dataset=base-comparateur-de-territoires&q=&rows=10000&facet=dep'
Paris_json_BaseData = requests.get(url1).json()
Paris_BaseData = Paris_json_BaseData['records']
Paris_BaseData[0] 

{'datasetid': 'base-comparateur-de-territoires',
 'recordid': 'cde03be66866c302ad7074170f911dc61a89d09f',
 'fields': {'p16_rp': 342.377850163,
  'p16_log': 385.939959189,
  'dece1116': 23.0,
  'p16_rsecocc': 3.84371550229,
  'med16': 24513.3333333,
  'codgeo': '77010',
  'etgz15': 15.0,
  'p16_men': 342.377850163,
  'p11_pop': 895.0,
  'libgeo': 'Aubepierre-Ozouer-le-Repos',
  'nais1116': 66.0,
  'superf': 26.83,
  'etaz15': 21.0,
  'etgu15': 49.0,
  'ettot15': 103.0,
  'p16_emplt': 154.129017012,
  'etbe15': 5.0,
  'reg': '11',
  'p16_emplt_sal': 121.355832507,
  'p16_act1564': 500.169381107,
  'etoq15': 8.0,
  'p11_emplt': 138.636729702,
  'naisd18': 11.0,
  'p16_logvac': 39.7183935236,
  'ettef115': 25.0,
  'geo_shape': {'type': 'Polygon',
   'coordinates': [[[2.9295822178, 48.6530321781],
     [2.9295119254, 48.6528081659],
     [2.9286216439, 48.6476212207],
     [2.9276894077, 48.6414915568],
     [2.9265149723, 48.639833079],
     [2.9243945184, 48.6382621409],
     [2.921935021

In [3]:
# define the dataframe columns
column_names = ['Postal Code', 'Commune', 'Departement', 'Size','Population2016','Jobs2016', 'geo2d'] 

# instantiate the dataframe
paris_basedata = pd.DataFrame(columns=column_names)

for data in Paris_BaseData:
    Postal_Code = data['fields']['codgeo']
    Commune= data['fields']['libgeo']
    Departement= data['fields']['dep']
    Size= data['fields']['superf']
    Population2016= data['fields']['p16_pop']
    Jobs2016= data['fields']['p16_emplt']
    geo2d= data['fields']['geo_point_2d']
    
    
    paris_basedata = paris_basedata.append({'Postal Code': Postal_Code,
                                          'Commune': Commune,
                                          'Departement': Departement,
                                          'Size': Size,
                                           'Population2016': Population2016,
                                           'Jobs2016': Jobs2016,
                                           'geo2d':geo2d
                                          
                                            
                                           }, ignore_index=True)

paris_basedata['Postal Code'].astype(str).astype(int) 
paris_basedata.head(100)


,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d
0,77010,Aubepierre-Ozouer-le-Repos,77,26.83,914.0,154.129017,"[48.6195655323, 2.90337876017]"
1,77032,Beton-Bazoches,77,18.32,894.0,163.260866,"[48.7075707227, 3.24574744908]"
2,77033,Bezalles,77,2.66,248.0,21.948645,"[48.6668992872, 3.24008063268]"
3,77035,Blennes,77,20.29,560.0,55.953791,"[48.2449628734, 3.0114100116]"
4,77042,Boissy-le-Châtel,77,9.95,3137.0,372.368140,"[48.8254382848, 3.13702822709]"
5,77045,Bougligny,77,16.31,731.0,101.360559,"[48.1991697304, 2.66724137339]"
6,77062,Carnetin,77,1.61,454.0,84.985551,"[48.9042770733, 2.69982266016]"
7,77079,Champagne-sur-Seine,77,7.28,6174.0,1113.342970,"[48.4087697375, 2.80758685861]"
8,77104,Châtres,77,15.13,667.0,905.695936,"[48.7130779585, 2.81582834461]"
9,77113,Chevru,77,13.94,1116.0,118.664885,"[48.7344347188, 3.19570344298]"


In [4]:
import math
paris_basedata['Latitude'] = paris_basedata['geo2d'].apply(pd.Series)[0]
paris_basedata['Longitude'] = paris_basedata['geo2d'].apply(pd.Series)[1]
paris_basedata['radius']=(paris_basedata['Size']/3.14)**(1/2)
dep_list = ['75','92','93','94']
paris_basedata = paris_basedata[paris_basedata.Departement.isin(dep_list)]
#paris_basedata.set_index("Postal Code", inplace = True) 

paris_basedata.head()


,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d,Latitude,Longitude,radius
96,92002,Antony,92,9.56,62210.0,22301.162683,"[48.7498603402, 2.29906093691]",48.749860,2.299061,1.744874
97,92009,Bois-Colombes,92,1.92,28323.0,12151.028582,"[48.9156889243, 2.26738167747]",48.915689,2.267382,0.781962
98,92014,Bourg-la-Reine,92,1.86,20531.0,5585.322901,"[48.7802725553, 2.3175926522]",48.780273,2.317593,0.769647
99,92019,Châtenay-Malabry,92,6.38,33016.0,7794.596177,"[48.7682137464, 2.26375687994]",48.768214,2.263757,1.425429
100,92060,Le Plessis-Robinson,92,3.43,29028.0,13751.475740,"[48.7802898374, 2.25979241613]",48.780290,2.259792,1.045159


<h3 align=left><font size = 5>1.2 Paris services</font></h3>

The second dataset provides basic statistics on services provided and number of restaurants

In [5]:
url2 = 'https://data.iledefrance.fr/api/records/1.0/search/?dataset=les-services-aux-particuliers-par-commune-ou-arrondissement-base-permanente-des-&q=&facet=departement&facet=libelle_de_commune&rows=10000'
Paris_json_services = requests.get(url2).json()
Paris_services = Paris_json_services['records']
Paris_services[0]

{'datasetid': 'les-services-aux-particuliers-par-commune-ou-arrondissement-base-permanente-des-',
 'recordid': 'ffec1a4fcec7d53bdae9dce55326aa6d2728d020',
 'fields': {'departement': 77,
  'conseil_de_prud_hommes': 0,
  'agence_de_services_specialises': 0,
  'agence_immobiliere': 2,
  'electricien': 6,
  'veterinaire': 0,
  'location_auto_utilitaires_legers': 0,
  'permanence_pole_emploi': 0,
  'pompes_funebres': 0,
  'population_2010': 3046,
  'blanchisserie_teinturerie': 1,
  'reparation_auto_et_de_materiel_agricole': 3,
  'canton_ville': 7728,
  'controle_technique_automobile': 0,
  'zone_d_emploi': 1105,
  'police': 0,
  'plombier_couvreur_chauffagiste': 8,
  'tresorerie': 0,
  'libelle_de_commune': 'Chaumes-en-Brie',
  'agence_de_proximite': 0,
  'relais_pole_emploi': 0,
  'unite_urbaine': 77111,
  'agence_postale_communale': 0,
  'ecole_de_conduite': 2,
  'arrondissement': 772,
  'cour_d_appel': 0,
  'agence_de_travail_temporaire': 0,
  'departement_commune': 77107,
  'relais_post

In [17]:
# define the dataframe columns
column_names = ['departement', 'Postal Code','agence_immobiliere','blanchisserie_teinturerie', 'police', 'soins_de_beaute','restaurant','bureau_de_poste'] 

# instantiate the dataframe
paris_services = pd.DataFrame(columns=column_names)

for data in Paris_services:
    departement = data['fields']['departement']
    commune = data['fields']['departement_commune']
    agence_immobiliere = data['fields']['agence_immobiliere'] 
    blanchisserie_teinturerie= data['fields']['blanchisserie_teinturerie']
    police= data['fields']['police']
    soins_de_beaute= data['fields']['soins_de_beaute']
    restaurant= data['fields']['restaurant']
    bureau_de_poste= data['fields']['bureau_de_poste']
   
    
    
    paris_services = paris_services.append({
                                        'departement' : departement,
                                          'Postal Code' : commune,
                                          'agence_immobiliere' : agence_immobiliere ,
                                           'blanchisserie_teinturerie' : blanchisserie_teinturerie,
    'police':police,
    'soins_de_beaute':soins_de_beaute,
    'restaurant':restaurant,
    'bureau_de_poste':bureau_de_poste,
                             
                                           }, ignore_index=True)


#paris_services.columns = paris_services.columns.map(lambda x: str(x) + '_s')
paris_services['Postal Code'].astype(str).astype(int) 
#paris_services.set_index("Postal Code", inplace = True) 
paris_services.head()

,departement,Postal Code,agence_immobiliere,blanchisserie_teinturerie,police,soins_de_beaute,restaurant,bureau_de_poste
0,77,77107,2,1,0,2,5,1
1,77,77109,0,0,0,2,4,1
2,77,77208,0,0,0,1,0,0
3,77,77235,0,0,0,0,0,0
4,77,77245,0,0,0,1,0,0


<h3 align=left><font size = 5>1.4 Foursquare data</font></h3>

Finally, we use Foursquare to fetch venues per category per commune in Paris

In [6]:
CLIENT_ID = 'TGAQELNBQMRCHSEBIWKYV2F4SGWAIEFFO5HYBPSDNP5VLAGM' # your Foursquare ID
CLIENT_SECRET = 'CCUDCPZMNOAAJ13NCXWVMZ4STKCDD1RDRKLEASINV2OAJ3MV' # your Foursquare Secret
LIMIT=100
VERSION = '20180605' # Foursquare API version

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
paris_venues = getNearbyVenues(names=paris_basedata['Commune'],
                                   latitudes=paris_basedata['Latitude'],
                                   longitudes=paris_basedata['Longitude'])
paris_venues.head()

Antony
Bois-Colombes
Bourg-la-Reine
Châtenay-Malabry
Le Plessis-Robinson
Bagnolet
Coubron
Montreuil
Tremblay-en-France
Villemomble
Ablon-sur-Seine
Boissy-Saint-Léger
L'Haÿ-les-Roses
Paris 1er Arrondissement
Paris 6e Arrondissement
Paris 12e Arrondissement
Paris 14e Arrondissement
Bagneux
Courbevoie
Malakoff
Sceaux
Vaucresson
Aulnay-sous-Bois
Le Bourget
Dugny
Neuilly-Plaisance
Rosny-sous-Bois
Charenton-le-Pont
Joinville-le-Pont
Ormesson-sur-Marne
Le Perreux-sur-Marne
Le Plessis-Trévise
Sucy-en-Brie
Valenton
Paris 19e Arrondissement
Paris 2e Arrondissement
Paris 8e Arrondissement
Paris 13e Arrondissement
Paris 20e Arrondissement
La Courneuve
Gournay-sur-Marne
Saint-Ouen-sur-Seine
Bonneuil-sur-Marne
Chevilly-Larue
Fontenay-sous-Bois
Orly
Villejuif
Villeneuve-le-Roi
Villeneuve-Saint-Georges
Boulogne-Billancourt
La Garenne-Colombes
Issy-les-Moulineaux
Levallois-Perret
Montrouge
Neuilly-sur-Seine
Aubervilliers
L'Île-Saint-Denis
Le Raincy
Villepinte
Villetaneuse
Alfortville
Arcueil
Champigny-

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Antony,48.74986,2.299061,Parc Heller,48.747377,2.293665,Park
1,Antony,48.74986,2.299061,Saveurs Tapas,48.749095,2.304329,Spanish Restaurant
2,Antony,48.74986,2.299061,La Beyrouthine,48.746814,2.303442,Restaurant
3,Antony,48.74986,2.299061,Les Galopins,48.749890,2.304591,French Restaurant
4,Antony,48.74986,2.299061,La Nostra,48.748921,2.304145,Fast Food Restaurant


Despite its international culture, French restaurants remain the dominant venue type in Paris

In [9]:
paris_venues.groupby('Venue Category')['Venue Category'].count().nlargest(300)


Venue Category
French Restaurant                 260
Hotel                             122
Italian Restaurant                 97
Supermarket                        93
Bakery                             83
Japanese Restaurant                74
Café                               70
Pizza Place                        51
Bistro                             49
Plaza                              49
Bar                                44
Coffee Shop                        40
Restaurant                         39
Park                               38
Asian Restaurant                   31
Wine Bar                           29
Sandwich Place                     28
Bus Stop                           27
Cocktail Bar                       26
Vietnamese Restaurant              24
Fast Food Restaurant               23
Chinese Restaurant                 22
Thai Restaurant                    21
Food & Drink Shop                  19
Burger Joint                       19
Sushi Restaurant                   

In [12]:
paris_venues.head(50)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Antony,48.749860,2.299061,Parc Heller,48.747377,2.293665,Park
1,Antony,48.749860,2.299061,Saveurs Tapas,48.749095,2.304329,Spanish Restaurant
2,Antony,48.749860,2.299061,La Beyrouthine,48.746814,2.303442,Restaurant
3,Antony,48.749860,2.299061,Les Galopins,48.749890,2.304591,French Restaurant
4,Antony,48.749860,2.299061,La Nostra,48.748921,2.304145,Fast Food Restaurant
5,Antony,48.749860,2.299061,Marché d'Antony,48.752872,2.303331,Farmers Market
6,Antony,48.749860,2.299061,New Dream,48.749426,2.303782,American Restaurant
7,Antony,48.749860,2.299061,Bistrot du Boucher,48.749936,2.303932,French Restaurant
8,Antony,48.749860,2.299061,Jokei,48.749511,2.304097,Sushi Restaurant
9,Antony,48.749860,2.299061,La Tour de Marrakech,48.748985,2.304008,Moroccan Restaurant


<a id='item1'></a>

<h2 align=left><font size = 5>2. Dataset preparation</font></h2>

We calculate jobs and population density per km2

In [18]:
paris_basedata.head(100)
paris_basedata['Population2016_density']=paris_basedata['Population2016']/paris_basedata['Size']
paris_basedata['Jobs2016_density']=paris_basedata['Jobs2016']/paris_basedata['Size']

paris_basedata.head(1000)

,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d,Latitude,Longitude,radius,Population2016_density,Jobs2016_density
96,92002,Antony,92,9.56,62210.0,22301.162683,"[48.7498603402, 2.29906093691]",48.749860,2.299061,1.744874,6507.322176,2332.757603
97,92009,Bois-Colombes,92,1.92,28323.0,12151.028582,"[48.9156889243, 2.26738167747]",48.915689,2.267382,0.781962,14751.562500,6328.660720
98,92014,Bourg-la-Reine,92,1.86,20531.0,5585.322901,"[48.7802725553, 2.3175926522]",48.780273,2.317593,0.769647,11038.172043,3002.861775
99,92019,Châtenay-Malabry,92,6.38,33016.0,7794.596177,"[48.7682137464, 2.26375687994]",48.768214,2.263757,1.425429,5174.921630,1221.723539
100,92060,Le Plessis-Robinson,92,3.43,29028.0,13751.475740,"[48.7802898374, 2.25979241613]",48.780290,2.259792,1.045159,8462.973761,4009.176601
101,93006,Bagnolet,93,2.57,35864.0,14261.915616,"[48.8694561887, 2.42256669611]",48.869456,2.422567,0.904694,13954.863813,5549.383508
102,93015,Coubron,93,4.14,4812.0,692.534903,"[48.9174973089, 2.57616543001]",48.917497,2.576165,1.148247,1162.318841,167.278962
103,93048,Montreuil,93,8.92,108402.0,54348.045276,"[48.8631652232, 2.4482059517]",48.863165,2.448206,1.685457,12152.690583,6092.830188
104,93073,Tremblay-en-France,93,22.44,35691.0,20550.976763,"[48.9784393812, 2.55500199052]",48.978439,2.555002,2.673293,1590.508021,915.818929
105,93077,Villemomble,93,4.04,30051.0,5206.996866,"[48.8843794006, 2.50918143824]",48.884379,2.509181,1.134295,7438.366337,1288.860610


In [19]:
paris_services.head()

,departement,Postal Code,agence_immobiliere,blanchisserie_teinturerie,police,soins_de_beaute,restaurant,bureau_de_poste
0,77,77107,2,1,0,2,5,1
1,77,77109,0,0,0,2,4,1
2,77,77208,0,0,0,1,0,0
3,77,77235,0,0,0,0,0,0
4,77,77245,0,0,0,1,0,0


In [20]:
paris_basedata['Postal Code'] = paris_basedata['Postal Code'].astype(int)
paris_services['Postal Code'] = paris_services['Postal Code'].astype(int)



In [38]:

paris_services.head()
#paris_basedata['Postal Code']=paris_basedata['Postal Code'].astype(int)
paris_basedata_merge = pd.merge(paris_basedata, paris_services, on=["Postal Code", "Postal Code"])


#df.join(other.set_index('key'), on='key')
 
paris_basedata_merge.head()


,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d,Latitude,Longitude,radius,Population2016_density,Jobs2016_density,departement,agence_immobiliere,blanchisserie_teinturerie,police,soins_de_beaute,restaurant,bureau_de_poste
0,92002,Antony,92,9.56,62210.0,22301.162683,"[48.7498603402, 2.29906093691]",48.749860,2.299061,1.744874,6507.322176,2332.757603,92,51,8,1,20,114,5
1,92009,Bois-Colombes,92,1.92,28323.0,12151.028582,"[48.9156889243, 2.26738167747]",48.915689,2.267382,0.781962,14751.562500,6328.660720,92,38,10,0,17,82,1
2,92014,Bourg-la-Reine,92,1.86,20531.0,5585.322901,"[48.7802725553, 2.3175926522]",48.780273,2.317593,0.769647,11038.172043,3002.861775,92,24,5,0,6,34,1
3,92019,Châtenay-Malabry,92,6.38,33016.0,7794.596177,"[48.7682137464, 2.26375687994]",48.768214,2.263757,1.425429,5174.921630,1221.723539,92,24,7,1,8,32,2
4,92060,Le Plessis-Robinson,92,3.43,29028.0,13751.475740,"[48.7802898374, 2.25979241613]",48.780290,2.259792,1.045159,8462.973761,4009.176601,92,21,3,0,11,28,2


We calculate service level per 1000 inhabitants. For restaurants, we take jobs + population as a reference

In [39]:
paris_basedata_merge['agence_immobiliere_dens']=1000*paris_basedata_merge['agence_immobiliere'].astype(int)/paris_basedata_merge['Population2016']
paris_basedata_merge['blanchisserie_teinturerie_dens']=1000*paris_basedata_merge['blanchisserie_teinturerie'].astype(int)/paris_basedata_merge['Population2016']
paris_basedata_merge['police_dens']=1000*paris_basedata_merge['police'].astype(int)/paris_basedata_merge['Population2016']
paris_basedata_merge['soins_de_beaute_dens']=1000*paris_basedata_merge['soins_de_beaute'].astype(int)/paris_basedata_merge['Population2016']
paris_basedata_merge['restaurant_dens']=1000*paris_basedata_merge['restaurant'].astype(int)/(paris_basedata_merge['Population2016']+paris_basedata_merge['Jobs2016'])
paris_basedata_merge['bureau_de_poste_dens']=1000*paris_basedata_merge['bureau_de_poste'].astype(int)/paris_basedata_merge['Population2016']




paris_basedata_merge.head()

,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d,Latitude,Longitude,radius,Population2016_density,Jobs2016_density,departement,agence_immobiliere,blanchisserie_teinturerie,police,soins_de_beaute,restaurant,bureau_de_poste,agence_immobiliere_dens,blanchisserie_teinturerie_dens,police_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens
0,92002,Antony,92,9.56,62210.0,22301.162683,"[48.7498603402, 2.29906093691]",48.749860,2.299061,1.744874,6507.322176,2332.757603,92,51,8,1,20,114,5,0.819804,0.128597,0.016075,0.321492,1.348934,0.080373
1,92009,Bois-Colombes,92,1.92,28323.0,12151.028582,"[48.9156889243, 2.26738167747]",48.915689,2.267382,0.781962,14751.562500,6328.660720,92,38,10,0,17,82,1,1.341666,0.353070,0.000000,0.600219,2.025991,0.035307
2,92014,Bourg-la-Reine,92,1.86,20531.0,5585.322901,"[48.7802725553, 2.3175926522]",48.780273,2.317593,0.769647,11038.172043,3002.861775,92,24,5,0,6,34,1,1.168964,0.243534,0.000000,0.292241,1.301868,0.048707
3,92019,Châtenay-Malabry,92,6.38,33016.0,7794.596177,"[48.7682137464, 2.26375687994]",48.768214,2.263757,1.425429,5174.921630,1221.723539,92,24,7,1,8,32,2,0.726920,0.212018,0.030288,0.242307,0.784110,0.060577
4,92060,Le Plessis-Robinson,92,3.43,29028.0,13751.475740,"[48.7802898374, 2.25979241613]",48.780290,2.259792,1.045159,8462.973761,4009.176601,92,21,3,0,11,28,2,0.723439,0.103348,0.000000,0.378944,0.654519,0.068899


In [40]:
paris_basedata_merge.dtypes

Postal Code                         int64
Commune                            object
Departement                        object
Size                              float64
Population2016                    float64
Jobs2016                          float64
geo2d                              object
Latitude                          float64
Longitude                         float64
radius                            float64
Population2016_density            float64
Jobs2016_density                  float64
departement                        object
agence_immobiliere                 object
blanchisserie_teinturerie          object
police                             object
soins_de_beaute                    object
restaurant                         object
bureau_de_poste                    object
agence_immobiliere_dens           float64
blanchisserie_teinturerie_dens    float64
police_dens                       float64
soins_de_beaute_dens              float64
restaurant_dens                   

<h2 align=left><font size = 5>3. Base data analysis</font></h2>

In [41]:
paris_basedata_merge.columns



Index(['Postal Code', 'Commune', 'Departement', 'Size', 'Population2016',
       'Jobs2016', 'geo2d', 'Latitude', 'Longitude', 'radius',
       'Population2016_density', 'Jobs2016_density', 'departement',
       'agence_immobiliere', 'blanchisserie_teinturerie', 'police',
       'soins_de_beaute', 'restaurant', 'bureau_de_poste',
       'agence_immobiliere_dens', 'blanchisserie_teinturerie_dens',
       'police_dens', 'soins_de_beaute_dens', 'restaurant_dens',
       'bureau_de_poste_dens'],
      dtype='object')

In [42]:
paris_basedata_merge_N = paris_basedata_merge.drop(['Postal Code','Commune', 'police_dens','Departement','Population2016','Jobs2016','Commune','geo2d' ,'Latitude','Longitude','radius','departement','agence_immobiliere','blanchisserie_teinturerie','police','soins_de_beaute','restaurant','bureau_de_poste'],axis=1)
paris_basedata_merge_N.head()


,Size,Population2016_density,Jobs2016_density,agence_immobiliere_dens,blanchisserie_teinturerie_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens
0,9.56,6507.322176,2332.757603,0.819804,0.128597,0.321492,1.348934,0.080373
1,1.92,14751.562500,6328.660720,1.341666,0.353070,0.600219,2.025991,0.035307
2,1.86,11038.172043,3002.861775,1.168964,0.243534,0.292241,1.301868,0.048707
3,6.38,5174.921630,1221.723539,0.726920,0.212018,0.242307,0.784110,0.060577
4,3.43,8462.973761,4009.176601,0.723439,0.103348,0.378944,0.654519,0.068899


In [45]:
paris_basedata_merge_N.dtypes

Size                              float64
Population2016_density            float64
Jobs2016_density                  float64
agence_immobiliere_dens           float64
blanchisserie_teinturerie_dens    float64
soins_de_beaute_dens              float64
restaurant_dens                   float64
bureau_de_poste_dens              float64
Clus_km                             int32
dtype: object

In [46]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
X = paris_basedata_merge_N.values[:,1:]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)
clusterNum = 5
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(Clus_dataSet)
labels = k_means.labels_
paris_basedata_merge_N["Clus_km"] = labels

paris_basedata_merge_N.head(1000)

,Size,Population2016_density,Jobs2016_density,agence_immobiliere_dens,blanchisserie_teinturerie_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens,Clus_km
0,9.56,6507.322176,2332.757603,0.819804,0.128597,0.321492,1.348934,0.080373,0
1,1.92,14751.562500,6328.660720,1.341666,0.353070,0.600219,2.025991,0.035307,4
2,1.86,11038.172043,3002.861775,1.168964,0.243534,0.292241,1.301868,0.048707,0
3,6.38,5174.921630,1221.723539,0.726920,0.212018,0.242307,0.784110,0.060577,0
4,3.43,8462.973761,4009.176601,0.723439,0.103348,0.378944,0.654519,0.068899,0
5,2.57,13954.863813,5549.383508,0.697078,0.223065,0.278831,2.513670,0.055766,0
6,4.14,1162.318841,167.278962,1.246883,0.207814,1.039069,0.908342,0.207814,2
7,8.92,12152.690583,6092.830188,0.654969,0.239848,0.332097,2.248847,0.055350,0
8,22.44,1590.508021,915.818929,0.672438,0.224146,0.252164,1.209061,0.056037,0
9,4.04,7438.366337,1288.860610,1.064856,0.199661,0.432598,1.758466,0.033277,0


In [47]:
#missing restaurants.
paris_basedata_merge_N.groupby('Clus_km').mean()

,Size,Population2016_density,Jobs2016_density,agence_immobiliere_dens,blanchisserie_teinturerie_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens
Clus_km,,,,,,,,
0,5.893333,6967.560665,2512.962495,0.757207,0.199988,0.374268,1.502430,0.070010
1,2.733750,27254.632779,27347.924766,4.104944,0.644216,1.264103,6.699232,0.129507
2,4.402500,1516.360489,1067.141225,1.015096,0.337094,0.742870,1.366642,0.273759
3,2.233333,12913.541291,46710.547258,21.662649,1.045720,3.010592,7.414339,0.249631
4,4.738929,20026.440196,11762.411550,1.861450,0.404132,0.607763,2.476392,0.067162


Clustering using base statistics highlights the following 5 clusters

Cluster 0: Large communes, medium density, limited services \
Cluster 1: Small communes, high density, residential, well-off \
Cluster 2: Medium communes, low density \
Cluster 3: Small communes, high-density of population and jobs \
Cluster 4: Medium communes, well off and residential


In [59]:
paris_basedata_merge['Clus_km'] = paris_basedata_merge_N['Clus_km']
paris_basedata_merge.head()

,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d,Latitude,Longitude,radius,Population2016_density,Jobs2016_density,departement,agence_immobiliere,blanchisserie_teinturerie,police,soins_de_beaute,restaurant,bureau_de_poste,agence_immobiliere_dens,blanchisserie_teinturerie_dens,police_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens,Clus_km,restaurant_dens_pred,restaurant_dens_var
18,92026,Courbevoie,92,4.17,81720.0,95277.137085,"[48.8983675206, 2.25515424075]",48.898368,2.255154,1.152400,19597.122302,22848.234313,92,120,23,1,49,236,5,1.468429,0.281449,0.012237,0.599608,1.333355,0.061185,4,3.687429,2.354074
101,94065,Rungis,94,4.20,5610.0,27219.354936,"[48.7495659946, 2.35212808498]",48.749566,2.352128,1.156538,1335.714286,6480.798794,94,4,3,0,5,32,2,0.713012,0.534759,0.000000,0.891266,0.974737,0.356506,2,3.237329,2.262592
89,75115,Paris 15e Arrondissement,75,8.48,233484.0,175585.406296,"[48.840040811, 2.29268840407]",48.840041,2.292688,1.643361,27533.490566,20705.826214,75,526,146,4,229,1122,15,2.252831,0.625311,0.017132,0.980795,2.742811,0.064244,4,4.541832,1.799021
106,92062,Puteaux,92,3.19,44662.0,77079.988390,"[48.8836186061, 2.23810959636]",48.883619,2.238110,1.007930,14000.626959,24163.005765,92,90,18,3,34,268,6,2.015136,0.403027,0.067171,0.761274,2.201377,0.134342,4,3.897884,1.696507
72,94067,Saint-Mandé,94,0.92,22731.0,7818.342823,"[48.8410208466, 2.41853928644]",48.841021,2.418539,0.541289,24707.608696,8498.198721,94,29,5,0,22,53,1,1.275791,0.219964,0.000000,0.967841,1.734898,0.043993,4,3.430587,1.695689


In [49]:
paris_basedata_merge_N.columns

Index(['Size', 'Population2016_density', 'Jobs2016_density',
       'agence_immobiliere_dens', 'blanchisserie_teinturerie_dens',
       'soins_de_beaute_dens', 'restaurant_dens', 'bureau_de_poste_dens',
       'Clus_km'],
      dtype='object')

In [52]:
X = paris_basedata_merge_N[['Size', 'Population2016_density', 'Jobs2016_density',
       'agence_immobiliere_dens', 
        'soins_de_beaute_dens',
       'bureau_de_poste_dens']] .values
y = paris_basedata_merge_N['restaurant_dens'].values
paris_basedata_merge_N.head()

,Size,Population2016_density,Jobs2016_density,agence_immobiliere_dens,blanchisserie_teinturerie_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens,Clus_km
0,9.56,6507.322176,2332.757603,0.819804,0.128597,0.321492,1.348934,0.080373,0
1,1.92,14751.562500,6328.660720,1.341666,0.353070,0.600219,2.025991,0.035307,4
2,1.86,11038.172043,3002.861775,1.168964,0.243534,0.292241,1.301868,0.048707,0
3,6.38,5174.921630,1221.723539,0.726920,0.212018,0.242307,0.784110,0.060577,0
4,3.43,8462.973761,4009.176601,0.723439,0.103348,0.378944,0.654519,0.068899,0


Let's try to predict number of restaurants per commune and try to see which commune might have a "restaurant deficit" based on a comparison between predicted value and real value

In [85]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
regr = linear_model.LinearRegression()

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
regr.fit (X_train, y_train)
y_hat= regr.predict(X_test)

print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y_test) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))

Residual sum of squares: 0.51
Variance score: 0.73


In [86]:
y_pred=regr.predict(X)

Then we reinject our predicted values and variances in teh main dataset

In [87]:
paris_basedata_merge['restaurant_dens_pred']=y_pred

paris_basedata_merge['restaurant_dens_var']=y_pred-paris_basedata_merge['restaurant_dens']


In [88]:
paris_basedata_merge.head()

,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d,Latitude,Longitude,radius,Population2016_density,Jobs2016_density,departement,agence_immobiliere,blanchisserie_teinturerie,police,soins_de_beaute,restaurant,bureau_de_poste,agence_immobiliere_dens,blanchisserie_teinturerie_dens,police_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens,Clus_km,restaurant_dens_pred,restaurant_dens_var
18,92026,Courbevoie,92,4.17,81720.0,95277.137085,"[48.8983675206, 2.25515424075]",48.898368,2.255154,1.152400,19597.122302,22848.234313,92,120,23,1,49,236,5,1.468429,0.281449,0.012237,0.599608,1.333355,0.061185,4,1.406062,0.072707
101,94065,Rungis,94,4.20,5610.0,27219.354936,"[48.7495659946, 2.35212808498]",48.749566,2.352128,1.156538,1335.714286,6480.798794,94,4,3,0,5,32,2,0.713012,0.534759,0.000000,0.891266,0.974737,0.356506,2,2.179874,1.205136
89,75115,Paris 15e Arrondissement,75,8.48,233484.0,175585.406296,"[48.840040811, 2.29268840407]",48.840041,2.292688,1.643361,27533.490566,20705.826214,75,526,146,4,229,1122,15,2.252831,0.625311,0.017132,0.980795,2.742811,0.064244,4,1.429965,-1.312845
106,92062,Puteaux,92,3.19,44662.0,77079.988390,"[48.8836186061, 2.23810959636]",48.883619,2.238110,1.007930,14000.626959,24163.005765,92,90,18,3,34,268,6,2.015136,0.403027,0.067171,0.761274,2.201377,0.134342,4,1.025147,-1.176230
72,94067,Saint-Mandé,94,0.92,22731.0,7818.342823,"[48.8410208466, 2.41853928644]",48.841021,2.418539,0.541289,24707.608696,8498.198721,94,29,5,0,22,53,1,1.275791,0.219964,0.000000,0.967841,1.734898,0.043993,4,1.577171,-0.157727


In [89]:
paris_basedata_merge.head()

,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d,Latitude,Longitude,radius,Population2016_density,Jobs2016_density,departement,agence_immobiliere,blanchisserie_teinturerie,police,soins_de_beaute,restaurant,bureau_de_poste,agence_immobiliere_dens,blanchisserie_teinturerie_dens,police_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens,Clus_km,restaurant_dens_pred,restaurant_dens_var
18,92026,Courbevoie,92,4.17,81720.0,95277.137085,"[48.8983675206, 2.25515424075]",48.898368,2.255154,1.152400,19597.122302,22848.234313,92,120,23,1,49,236,5,1.468429,0.281449,0.012237,0.599608,1.333355,0.061185,4,1.406062,0.072707
101,94065,Rungis,94,4.20,5610.0,27219.354936,"[48.7495659946, 2.35212808498]",48.749566,2.352128,1.156538,1335.714286,6480.798794,94,4,3,0,5,32,2,0.713012,0.534759,0.000000,0.891266,0.974737,0.356506,2,2.179874,1.205136
89,75115,Paris 15e Arrondissement,75,8.48,233484.0,175585.406296,"[48.840040811, 2.29268840407]",48.840041,2.292688,1.643361,27533.490566,20705.826214,75,526,146,4,229,1122,15,2.252831,0.625311,0.017132,0.980795,2.742811,0.064244,4,1.429965,-1.312845
106,92062,Puteaux,92,3.19,44662.0,77079.988390,"[48.8836186061, 2.23810959636]",48.883619,2.238110,1.007930,14000.626959,24163.005765,92,90,18,3,34,268,6,2.015136,0.403027,0.067171,0.761274,2.201377,0.134342,4,1.025147,-1.176230
72,94067,Saint-Mandé,94,0.92,22731.0,7818.342823,"[48.8410208466, 2.41853928644]",48.841021,2.418539,0.541289,24707.608696,8498.198721,94,29,5,0,22,53,1,1.275791,0.219964,0.000000,0.967841,1.734898,0.043993,4,1.577171,-0.157727


In [90]:
paris_basedata_merge.sort_values(by=['restaurant_dens_var'], inplace=True, ascending=False)
paris_basedata_merge.head(20)

,Postal Code,Commune,Departement,Size,Population2016,Jobs2016,geo2d,Latitude,Longitude,radius,Population2016_density,Jobs2016_density,departement,agence_immobiliere,blanchisserie_teinturerie,police,soins_de_beaute,restaurant,bureau_de_poste,agence_immobiliere_dens,blanchisserie_teinturerie_dens,police_dens,soins_de_beaute_dens,restaurant_dens,bureau_de_poste_dens,Clus_km,restaurant_dens_pred,restaurant_dens_var
63,94019,Chennevières-sur-Marne,94,5.27,18396.0,5917.694616,"[48.7978886949, 2.54178280739]",48.797889,2.541783,1.295509,3490.702087,1122.902204,94,19,4,1,16,33,2,1.032833,0.217439,0.054360,0.869754,1.357260,0.108719,0,8.387742,7.030482
4,92060,Le Plessis-Robinson,92,3.43,29028.0,13751.475740,"[48.7802898374, 2.25979241613]",48.780290,2.259792,1.045159,8462.973761,4009.176601,92,21,3,0,11,28,2,0.723439,0.103348,0.000000,0.378944,0.654519,0.068899,0,7.563840,6.909320
129,92033,Garches,92,2.69,17663.0,5536.053303,"[48.8456238029, 2.18800429561]",48.845624,2.188004,0.925574,6566.171004,2058.012380,92,35,5,0,8,18,1,1.981543,0.283078,0.000000,0.452924,0.775894,0.056616,0,5.794346,5.018452
43,94021,Chevilly-Larue,94,4.22,19347.0,7420.980220,"[48.7667116171, 2.35261812093]",48.766712,2.352618,1.159288,4584.597156,1758.526118,94,17,3,1,3,44,1,0.878689,0.155063,0.051688,0.155063,1.643755,0.051688,0,5.521196,3.877441
3,92019,Châtenay-Malabry,92,6.38,33016.0,7794.596177,"[48.7682137464, 2.26375687994]",48.768214,2.263757,1.425429,5174.921630,1221.723539,92,24,7,1,8,32,2,0.726920,0.212018,0.030288,0.242307,0.784110,0.060577,0,4.253750,3.469640
12,94038,L'Haÿ-les-Roses,94,3.90,31189.0,5147.873054,"[48.776029777, 2.33737630173]",48.776030,2.337376,1.114468,7997.179487,1319.967450,94,13,4,1,16,33,1,0.416814,0.128250,0.032063,0.513001,0.908168,0.032063,0,3.744999,2.836831
40,93033,Gournay-sur-Marne,93,1.68,6861.0,1000.282813,"[48.8609572803, 2.57591532683]",48.860957,2.575915,0.731459,4083.928571,595.406436,93,13,1,0,5,20,1,1.894768,0.145751,0.000000,0.728757,2.544114,0.145751,0,5.218636,2.674522
30,94058,Le Perreux-sur-Marne,94,3.96,33729.0,5718.382823,"[48.8421812372, 2.50400643241]",48.842181,2.504006,1.123008,8517.424242,1444.036067,94,53,10,0,27,76,2,1.571348,0.296481,0.000000,0.800498,1.926617,0.059296,0,4.541832,2.615215
21,92076,Vaucresson,92,3.08,8628.0,2759.565321,"[48.8419841105, 2.1614035779]",48.841984,2.161404,0.990400,2801.298701,895.962766,92,18,2,0,5,10,1,2.086231,0.231803,0.000000,0.579509,0.878151,0.115902,0,3.483660,2.605509
85,94028,Créteil,94,11.46,89392.0,53177.276646,"[48.783689162, 2.45498047131]",48.783689,2.454980,1.910414,7800.349040,4640.251016,94,52,18,3,36,167,5,0.581708,0.201360,0.033560,0.402721,1.171360,0.055933,0,3.686354,2.514993


In [61]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [91]:
paris_basedata_merge.columns

Index(['Postal Code', 'Commune', 'Departement', 'Size', 'Population2016',
       'Jobs2016', 'geo2d', 'Latitude', 'Longitude', 'radius',
       'Population2016_density', 'Jobs2016_density', 'departement',
       'agence_immobiliere', 'blanchisserie_teinturerie', 'police',
       'soins_de_beaute', 'restaurant', 'bureau_de_poste',
       'agence_immobiliere_dens', 'blanchisserie_teinturerie_dens',
       'police_dens', 'soins_de_beaute_dens', 'restaurant_dens',
       'bureau_de_poste_dens', 'Clus_km', 'restaurant_dens_pred',
       'restaurant_dens_var'],
      dtype='object')

In [95]:
Geo_view_BaseClusters = paris_basedata_merge[['Postal Code', 'Commune', 'Latitude','Longitude','Clus_km','restaurant_dens_var']].copy()
Geo_view_BaseClusters.head(200)

,Postal Code,Commune,Latitude,Longitude,Clus_km,restaurant_dens_var
63,94019,Chennevières-sur-Marne,48.797889,2.541783,0,7.030482
4,92060,Le Plessis-Robinson,48.780290,2.259792,0,6.909320
129,92033,Garches,48.845624,2.188004,0,5.018452
43,94021,Chevilly-Larue,48.766712,2.352618,0,3.877441
3,92019,Châtenay-Malabry,48.768214,2.263757,0,3.469640
12,94038,L'Haÿ-les-Roses,48.776030,2.337376,0,2.836831
40,93033,Gournay-sur-Marne,48.860957,2.575915,0,2.674522
30,94058,Le Perreux-sur-Marne,48.842181,2.504006,0,2.615215
21,92076,Vaucresson,48.841984,2.161404,0,2.605509
85,94028,Créteil,48.783689,2.454980,0,2.514993


In [78]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [82]:
Lat_paris
clusterNum

5

In [83]:
# create map
Lat_paris = 48.856600
Long_paris = 2.352200
map_clusters_base = folium.Map(location=[Lat_paris, Long_paris], zoom_start=12)

# set color scheme for the clusters
x = np.arange(clusterNum)
ys = [i + x + (i*x)**2 for i in range(clusterNum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Geo_view_BaseClusters['Latitude'], Geo_view_BaseClusters['Longitude'], Geo_view_BaseClusters['Commune'], Geo_view_BaseClusters['Clus_km']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_base)

map_clusters_base

Cluster 0: Large communes, medium density, limited services \
Cluster 1: Small communes, high density, residential, well-off \
Cluster 2: Medium communes, low density \
Cluster 3: Small communes, high-density of population and jobs \
Cluster 4: Medium communes, well off and residential

In [100]:
Geo_view_BaseClusters_Cluster1=Geo_view_BaseClusters[Geo_view_BaseClusters['Clus_km']==1].sort_values(by=['restaurant_dens_var'], ascending=False)
Geo_view_BaseClusters_Cluster3=Geo_view_BaseClusters[Geo_view_BaseClusters['Clus_km']==3].sort_values(by=['restaurant_dens_var'], ascending=False)
Geo_view_BaseClusters_Cluster4=Geo_view_BaseClusters[Geo_view_BaseClusters['Clus_km']==4].sort_values(by=['restaurant_dens_var'], ascending=False)



In [101]:
Geo_view_BaseClusters_Cluster4.head()

,Postal Code,Commune,Latitude,Longitude,Clus_km,restaurant_dens_var
51,92040,Issy-les-Moulineaux,48.823121,2.264459,4,2.250976
50,92035,La Garenne-Colombes,48.907055,2.244982,4,0.976939
54,92051,Neuilly-sur-Seine,48.885564,2.266246,4,0.970813
38,75120,Paris 20e Arrondissement,48.863526,2.401212,4,0.786951
1,92009,Bois-Colombes,48.915689,2.267382,4,0.249022


In [102]:
Geo_view_BaseClusters_Cluster3.head()

,Postal Code,Commune,Latitude,Longitude,Clus_km,restaurant_dens_var
36,75108,Paris 8e Arrondissement,48.872785,2.312531,3,-0.338084
35,75102,Paris 2e Arrondissement,48.868218,2.342690,3,-7.073814
13,75101,Paris 1er Arrondissement,48.862526,2.336301,3,-7.323700


In [103]:
Geo_view_BaseClusters_Cluster1.head()

,Postal Code,Commune,Latitude,Longitude,Clus_km,restaurant_dens_var
14,75106,Paris 6e Arrondissement,48.849112,2.332847,1,0.829686
140,75105,Paris 5e Arrondissement,48.844409,2.350498,1,-2.808803
125,75104,Paris 4e Arrondissement,48.854287,2.357596,1,-3.321588
127,75117,Paris 17e Arrondissement,48.887374,2.306813,1,-3.408139
142,75111,Paris 11e Arrondissement,48.859110,2.380177,1,-3.566422


<h2 align=left><font size = 5>4. Cluster per venue</font></h2>

In [113]:
paris_venues_target_paris6 = paris_venues[paris_venues['Neighborhood']=='Paris 6e Arrondissement']
paris_venues_target_issy = paris_venues[paris_venues['Neighborhood']=='Issy-les-Moulineaux']
paris_venues_target = pd.concat([paris_venues_target_paris6, paris_venues_target_issy])
paris_venues_target.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
192,Paris 6e Arrondissement,48.849112,2.332847,Place Saint-Sulpice,48.850823,2.333323,Plaza
193,Paris 6e Arrondissement,48.849112,2.332847,Sadaharu Aoki | 青木定治,48.848013,2.330366,Dessert Shop
194,Paris 6e Arrondissement,48.849112,2.332847,Le Petit Lux,48.848227,2.331290,Sandwich Place
195,Paris 6e Arrondissement,48.849112,2.332847,Pizza Chic,48.850274,2.330766,Pizza Place
196,Paris 6e Arrondissement,48.849112,2.332847,Pierre Hermé,48.851532,2.332766,Pastry Shop


In [114]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues_target[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighborhood'] = paris_venues_target['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Athletics & Sports,Bakery,Bistro,Bookstore,Brasserie,Burger Joint,Bus Stop,Café,Cajun / Creole Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Dessert Shop,Electronics Store,Falafel Restaurant,Fountain,French Restaurant,Garden,Greek Restaurant,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lebanese Restaurant,Market,Miscellaneous Shop,New American Restaurant,Outdoor Sculpture,Pastry Shop,Pharmacy,Pizza Place,Playground,Plaza,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Stadium,Supermarket,Tailor Shop,Tea Room,Tennis Court,Theater,Tram Station,Wine Bar
192,Paris 6e Arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
193,Paris 6e Arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
194,Paris 6e Arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
195,Paris 6e Arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
196,Paris 6e Arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [116]:
paris_grouped = paris_onehot.groupby('Neighborhood').mean().reset_index()
paris_grouped

,Neighborhood,American Restaurant,Art Gallery,Athletics & Sports,Bakery,Bistro,Bookstore,Brasserie,Burger Joint,Bus Stop,Café,Cajun / Creole Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Dessert Shop,Electronics Store,Falafel Restaurant,Fountain,French Restaurant,Garden,Greek Restaurant,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lebanese Restaurant,Market,Miscellaneous Shop,New American Restaurant,Outdoor Sculpture,Pastry Shop,Pharmacy,Pizza Place,Playground,Plaza,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Stadium,Supermarket,Tailor Shop,Tea Room,Tennis Court,Theater,Tram Station,Wine Bar
0,Issy-les-Moulineaux,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.04,0.04,0.04,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.080000,0.000000,0.000000,0.08,0.080000,0.000000,0.04,0.160000,0.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.080000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000
1,Paris 6e Arrondissement,0.015385,0.015385,0.030769,0.030769,0.030769,0.030769,0.00,0.00,0.00,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.00,0.030769,0.107692,0.015385,0.015385,0.00,0.015385,0.030769,0.00,0.076923,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.030769,0.015385,0.015385,0.015385,0.015385,0.030769,0.015385,0.015385,0.015385,0.015385,0.00,0.015385,0.015385,0.015385,0.015385,0.015385,0.00,0.061538


In [117]:
num_top_venues = 5

for hood in paris_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Issy-les-Moulineaux----
                  venue  freq
0    Italian Restaurant  0.16
1   Japanese Restaurant  0.12
2  Gym / Fitness Center  0.08
3                 Hotel  0.08
4           Supermarket  0.08


----Paris 6e Arrondissement----
                venue  freq
0   French Restaurant  0.11
1  Italian Restaurant  0.08
2            Wine Bar  0.06
3              Bakery  0.03
4           Bookstore  0.03




In [123]:
paris_venues_target_paris6.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
192,Paris 6e Arrondissement,48.849112,2.332847,Place Saint-Sulpice,48.850823,2.333323,Plaza
193,Paris 6e Arrondissement,48.849112,2.332847,Sadaharu Aoki | 青木定治,48.848013,2.330366,Dessert Shop
194,Paris 6e Arrondissement,48.849112,2.332847,Le Petit Lux,48.848227,2.331290,Sandwich Place
195,Paris 6e Arrondissement,48.849112,2.332847,Pizza Chic,48.850274,2.330766,Pizza Place
196,Paris 6e Arrondissement,48.849112,2.332847,Pierre Hermé,48.851532,2.332766,Pastry Shop


In [133]:
Lat_paris6=48.848227
Long_paris6=2.331290

map_clusters_paris6 = folium.Map(location=[Lat_paris6, Long_paris6], zoom_start=16)



# add markers to the map
markers_colors = []
for lat, lng, label in zip(paris_venues_target_paris6['Venue Latitude'], paris_venues_target_paris6['Venue Longitude'], paris_venues_target_paris6['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters_paris6)

map_clusters_paris6


In [135]:
paris_venues_target_issy.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
875,Issy-les-Moulineaux,48.823121,2.264459,MurMur,48.823489,2.262604,Gym / Fitness Center
876,Issy-les-Moulineaux,48.823121,2.264459,Laura Traiteur,48.826148,2.265917,Falafel Restaurant
877,Issy-les-Moulineaux,48.823121,2.264459,La Factory,48.823200,2.260660,Burger Joint
878,Issy-les-Moulineaux,48.823121,2.264459,CMG Sports Club One Issy,48.826129,2.260100,Gym / Fitness Center
879,Issy-les-Moulineaux,48.823121,2.264459,Restaurant du Stade,48.825232,2.264355,French Restaurant


In [137]:
Lat_issy=48.823121
Long_issy=2.264459

map_clusters_issy = folium.Map(location=[Lat_issy, Long_issy], zoom_start=16)



# add markers to the map
markers_colors = []
for lat, lng, label in zip(paris_venues_target_issy['Venue Latitude'], paris_venues_target_issy['Venue Longitude'], paris_venues_target_issy['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters_issy)

map_clusters_issy